# SOTIF Uncertainty Evaluation for LiDAR Object Detection

**Uncertainty Evaluation to Support Safety of the Intended Functionality Analysis for Identifying Performance Insufficiencies in ML-Based LiDAR Object Detection**

*Milin Patel and Rolf Jung, Kempten University of Applied Sciences*

---

This notebook demonstrates the complete evaluation methodology from the paper. It implements all five stages of the pipeline and produces the four SOTIF artefacts mapped to ISO 21448 clauses.

**What this notebook does:**
1. Generates synthetic ensemble outputs matching the paper's statistical properties (or loads real data)
2. Computes three uncertainty indicators per detection (Stage 2)
3. Determines correctness labels (Stage 3)
4. Computes discrimination, calibration, and operating characteristics (Stage 4)
5. Produces SOTIF analysis artefacts: TC ranking, frame flags, acceptance gates (Stage 5)
6. Generates all figures from the paper

**No GPU required.** The evaluation pipeline runs on CPU. For training ensemble models on real data, see the `scripts/` directory.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection/blob/main/notebooks/SOTIF_Uncertainty_Evaluation_Demo.ipynb)

## 0. Setup

Install the package and dependencies.

In [ ]:
# Clone the repository (if running in Colab)
import os
if 'google.colab' in str(get_ipython()):
    if not os.path.exists('SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection'):
        !git clone https://github.com/milinpatel07/SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection.git
    os.chdir('SOTIF_Uncertainty_Evaluation_Lidar_Object_Detection')
    !pip install -q -e .
else:
    # Running locally: ensure you're in the repo root
    pass

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

print(f"NumPy version: {np.__version__}")
print("Setup complete.")

## 1. Generate / Load Ensemble Data

The demo uses synthetic data that matches the statistical properties reported in the paper:
- **465 proposals**: 135 true positives (TP), 330 false positives (FP)
- **6 ensemble members** (SECOND detectors with different random seeds)
- **22 environmental configurations** across 101 frames
- **4 triggering condition categories**: heavy rain, night, fog/visibility, other

To use real ensemble outputs, replace this cell with code that loads your prediction files (see Section 8).

In [ ]:
from sotif_uncertainty.demo_data import generate_demo_dataset, get_individual_member_aps

# Generate synthetic ensemble data
data = generate_demo_dataset(
    n_tp=135,
    n_fp=330,
    K=6,
    n_frames=101,
    seed=42,
)

scores = data['scores']           # (465, 6) confidence scores per member
boxes = data['boxes']             # (465, 6, 7) bounding boxes per member
labels = data['labels']           # (465,) correctness: 1=TP, 0=FP
frame_ids = data['frame_ids']     # (465,) frame assignment
conditions = data['conditions']   # (465,) triggering condition category

print(f"Dataset: {len(labels)} proposals ({np.sum(labels==1)} TP, {np.sum(labels==0)} FP)")
print(f"Ensemble size: K={data['K']}")
print(f"Frames: {data['n_frames']}")
print(f"Conditions: {np.unique(conditions)}")

# Individual member performance (Table 1)
member_aps = get_individual_member_aps()
print(f"\nIndividual Member BEV AP (IoU >= 0.5):")
for name, ap in member_aps.items():
    print(f"  Member {name}: {ap:.2f}%")

## 2. Stage 2: Compute Uncertainty Indicators

Three uncertainty indicators per proposal (Section 3.3, Table 2):

| Indicator | Uncertainty Type | Safety Concern | SOTIF Activity |
|-----------|-----------------|----------------|----------------|
| Mean confidence $\bar{s}_j$ | Existence | False/missed detection | PI identification (Cl. 7) |
| Confidence variance $\sigma^2_{s,j}$ | Epistemic | Unknown operating condition | TC identification (Cl. 7) |
| Geometric disagreement $d_{\text{iou},j}$ | Localisation | Incorrect distance estimate | PI characterisation (Cl. 7) |

In [ ]:
from sotif_uncertainty.uncertainty import compute_all_indicators

# Compute all three uncertainty indicators
indicators = compute_all_indicators(scores, boxes)

mean_conf = indicators['mean_confidence']       # Eq. 1
conf_var = indicators['confidence_variance']     # Eq. 2
geo_disagree = indicators['geometric_disagreement']  # Eq. 3

# Summary statistics
print("=" * 60)
print("UNCERTAINTY INDICATOR SUMMARY")
print("=" * 60)
for name, vals, lab in [
    ("Mean Confidence", mean_conf, labels),
    ("Confidence Variance", conf_var, labels),
    ("Geometric Disagreement", geo_disagree, labels),
]:
    tp_vals = vals[lab == 1]
    fp_vals = vals[lab == 0]
    print(f"\n{name}:")
    print(f"  TP: mean={np.mean(tp_vals):.4f}, median={np.median(tp_vals):.4f}, "
          f"80th pct={np.percentile(tp_vals, 80):.5f}")
    print(f"  FP: mean={np.mean(fp_vals):.4f}, median={np.median(fp_vals):.4f}, "
          f"80th pct={np.percentile(fp_vals, 80):.5f}")
    print(f"  Separation factor (80th pct FP / 80th pct TP): "
          f"{np.percentile(fp_vals, 80) / max(np.percentile(tp_vals, 80), 1e-10):.2f}x")

## 3. Stage 4: Discrimination Metrics

**Key question**: *Does uncertainty identify which detections are incorrect?*

This is the prerequisite for all SOTIF uses of uncertainty. If discrimination is poor, the subsequent analysis is not supported.

- **AUROC** measures ranking quality (1.0 = perfect, 0.5 = random)
- **AURC** measures residual risk as a function of coverage

In [ ]:
from sotif_uncertainty.metrics import (
    compute_auroc, compute_auroc_with_curve, compute_aurc, compute_all_metrics
)

# Compute all metrics
metrics = compute_all_metrics(mean_conf, conf_var, geo_disagree, labels)

disc = metrics['discrimination']
print("=" * 60)
print("DISCRIMINATION: AUROC for TP vs. FP Separation (Table 3)")
print("=" * 60)
print(f"  Mean Confidence (s_bar):          AUROC = {disc['auroc_mean_confidence']:.3f}")
print(f"  Confidence Variance (sigma^2_s):   AUROC = {disc['auroc_confidence_variance']:.3f}")
print(f"  Geometric Disagreement (d_iou):    AUROC = {disc['auroc_geometric_disagreement']:.3f}")

print(f"\n  Interpretation:")
print(f"  - Mean confidence achieves near-perfect discrimination")
print(f"  - Variance captures epistemic uncertainty (model disagreement)")
print(f"  - Geometric disagreement captures localisation uncertainty")
print(f"  - The three indicators are complementary (Table 2 in paper)")

In [ ]:
# Plot ROC curves for all three indicators
from sotif_uncertainty.visualization import plot_roc_curves

# Get ROC curve data for each indicator
auroc_conf, fpr_conf, tpr_conf = compute_auroc_with_curve(
    mean_conf, labels, higher_is_correct=True
)
auroc_var, fpr_var, tpr_var = compute_auroc_with_curve(
    -conf_var, labels, higher_is_correct=True  # negate: lower var = correct
)
auroc_geo, fpr_geo, tpr_geo = compute_auroc_with_curve(
    -geo_disagree, labels, higher_is_correct=True  # negate: lower disagreement = correct
)

roc_data = {
    'Mean Confidence': (fpr_conf, tpr_conf, auroc_conf),
    'Confidence Variance': (fpr_var, tpr_var, auroc_var),
    'Geometric Disagreement': (fpr_geo, tpr_geo, auroc_geo),
}

fig = plot_roc_curves(roc_data)
plt.show()

## 4. Stage 4: Calibration Metrics

**Key question**: *Do confidence values match observed accuracy?*

- **ECE**: Expected Calibration Error (Eq. 4)
- **NLL**: Negative Log-Likelihood (Eq. 5)
- **Brier Score**: Mean squared error between confidence and label (Eq. 6)

**Important**: High discrimination and moderate calibration can coexist (Section 6.2). Acceptance gates depend on *ranking*, not absolute calibration.

In [ ]:
cal = metrics['calibration']

print("=" * 60)
print("CALIBRATION AND SCORING METRICS (Table 4)")
print("=" * 60)
print(f"  ECE:         {cal['ece']:.3f}")
print(f"  NLL:         {cal['nll']:.3f}")
print(f"  Brier Score: {cal['brier']:.3f}")
print(f"  AURC:        {metrics['risk_coverage']['aurc']:.3f}")

print(f"\n  Interpretation (Section 6.2):")
print(f"  - ECE of {cal['ece']:.3f} means confidence deviates from accuracy by ~{cal['ece']*100:.0f} pp")
print(f"  - Acceptance gates depend on RANKING (AUROC), not calibration (ECE)")
print(f"  - If downstream modules interpret confidence probabilistically,")
print(f"    post-hoc calibration (temperature scaling) can be applied")

In [ ]:
# Reliability Diagram (Figure 5)
from sotif_uncertainty.visualization import plot_reliability_diagram

fig = plot_reliability_diagram(
    cal['bin_accuracies'],
    cal['bin_confidences'],
    cal['bin_counts'],
    cal['ece'],
)
plt.show()

In [ ]:
# Risk-Coverage Curve (Figure 6)
from sotif_uncertainty.visualization import plot_risk_coverage

rc = metrics['risk_coverage']
fig = plot_risk_coverage(rc['coverages'], rc['risks'], rc['aurc'])
plt.show()

# Find zero-risk coverage
zero_risk_idx = 0
for i, r in enumerate(rc['risks']):
    if r > 0:
        break
    zero_risk_idx = i
print(f"Zero-risk region: coverage up to {rc['coverages'][zero_risk_idx]:.3f}")
print(f"(i.e., the top {rc['coverages'][zero_risk_idx]*100:.1f}% most confident detections are all correct)")

## 5. Uncertainty Landscape Visualisation

Scatter plot of mean confidence vs. confidence variance, coloured by correctness label (Figure 7).

**Expected pattern**: TP cluster at high confidence / low variance. FP spread across low confidence / high variance.

In [ ]:
from sotif_uncertainty.visualization import plot_scatter_confidence_variance

fig = plot_scatter_confidence_variance(mean_conf, conf_var, labels)
plt.show()

## 6. Stage 4: Operating Points (Acceptance Gates)

**Key question**: *What is the tradeoff between retained capability and retained error?*

An acceptance gate (Eq. 7) filters detections:

$$G(\bar{s}_j, \sigma^2_{s,j}, d_{\text{iou},j}) = [\bar{s}_j \geq \tau_s] \wedge [\sigma^2_{s,j} \leq \tau_v] \wedge [d_{\text{iou},j} \leq \tau_d]$$

ISO 21448, Clause 11 requires documented acceptance criteria. The operating point table provides this.

In [ ]:
from sotif_uncertainty.sotif_analysis import compute_operating_points, find_optimal_gate

# Compute operating points with confidence-only and multi-indicator gates
# Confidence-only gates
conf_only_points = compute_operating_points(
    mean_conf, conf_var, geo_disagree, labels,
    tau_s_range=np.array([0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90]),
)

# Multi-indicator gates
multi_points = compute_operating_points(
    mean_conf, conf_var, geo_disagree, labels,
    tau_s_range=np.array([0.60, 0.65, 0.70]),
    tau_v_range=np.array([0.002, 0.003]),
)

all_points = conf_only_points + multi_points

# Display operating point table (Table 6)
print("=" * 75)
print("OPERATING POINTS TABLE (Table 6 / Clause 11 Input)")
print("=" * 75)
print(f"{'Gate':<35} {'Cov.':>6} {'Ret.':>5} {'FP':>4} {'FAR':>8}")
print("-" * 75)

# Show key operating points
key_gates = [
    "s>=0.60", "s>=0.65", "s>=0.70", "s>=0.85",
    "s>=0.65 & var<=0.0020", "s>=0.60 & var<=0.0020",
]
for point in all_points:
    if point['gate'] in key_gates or point['coverage'] > 0:
        print(f"{point['gate']:<35} {point['coverage']:>6.3f} {point['retained']:>5d} "
              f"{point['fp']:>4d} {point['far']:>8.3f}")

# Find optimal gate for zero FAR
print("\n" + "-" * 75)
optimal = find_optimal_gate(
    mean_conf, conf_var, geo_disagree, labels,
    alpha=0.0,  # Zero false acceptance
    tau_s_range=np.arange(0.50, 0.91, 0.05),
    tau_v_range=np.array([np.inf, 0.002, 0.003]),
)
if optimal:
    print(f"Optimal gate (FAR=0): {optimal['gate']}")
    print(f"  Coverage: {optimal['coverage']:.3f} ({optimal['retained']} detections retained)")
    print(f"  All retained detections are correct (FAR=0)")

In [ ]:
# Operating points visualisation
from sotif_uncertainty.visualization import plot_operating_points_comparison

fig = plot_operating_points_comparison(all_points)
plt.show()

### 6.1 Value of Multi-Indicator Gates (Section 6.3)

A detection with $\bar{s}=0.67$ and $\sigma^2_s=0.001$ indicates member **agreement** (likely correct).  
A detection with $\bar{s}=0.67$ and $\sigma^2_s=0.005$ indicates member **disagreement** (likely incorrect).

The multi-indicator gate retains the first and rejects the second; a confidence-only gate at 0.65 retains both.

In [ ]:
from sotif_uncertainty.sotif_analysis import acceptance_gate, compute_coverage_far

# Compare: confidence-only at 0.65 vs multi-indicator at 0.65+var<=0.002
accepted_conf = acceptance_gate(mean_conf, conf_var, geo_disagree, tau_s=0.65)
accepted_multi = acceptance_gate(mean_conf, conf_var, geo_disagree, tau_s=0.65, tau_v=0.002)

cov_c, far_c, ret_c, fp_c = compute_coverage_far(accepted_conf, labels)
cov_m, far_m, ret_m, fp_m = compute_coverage_far(accepted_multi, labels)

print("Multi-indicator gate comparison:")
print(f"  Confidence-only (s>=0.65):         {ret_c} retained, {fp_c} FP, FAR={far_c:.3f}")
print(f"  Multi-indicator (s>=0.65, v<=0.002): {ret_m} retained, {fp_m} FP, FAR={far_m:.3f}")
print(f"  Variance filter removes {ret_c - ret_m} detections, reducing FP by {fp_c - fp_m}")

## 7. Stage 5: SOTIF Analysis Artefacts

### 7.1 Triggering Condition Identification (Clause 7)

Rank conditions by FP share and mean uncertainty. If both rankings agree, uncertainty analysis independently identifies the triggering conditions that error analysis identifies.

In [ ]:
from sotif_uncertainty.sotif_analysis import rank_triggering_conditions

tc_results = rank_triggering_conditions(conditions, labels, mean_conf, conf_var)

print("=" * 70)
print("TRIGGERING CONDITION RANKING (Table 7 / Clause 7)")
print("=" * 70)
print(f"{'Condition':<18} {'FP Share':>10} {'Mean s (FP)':>12} {'Mean var (FP)':>14} {'TP':>5} {'FP':>5}")
print("-" * 70)
for tc in tc_results:
    print(f"{tc['condition']:<18} {tc['fp_share']:>10.1%} {tc['mean_conf_fp']:>12.2f} "
          f"{tc['mean_var_fp']:>14.4f} {tc['tp_count']:>5d} {tc['fp_count']:>5d}")

print(f"\nInterpretation:")
print(f"  - Heavy rain and night account for ~67% of all false positives")
print(f"  - Ranking by FP share matches ranking by mean FP variance")
print(f"  - This means uncertainty statistics alone can identify which")
print(f"    conditions are most problematic, even without ground truth")

In [ ]:
# Triggering condition visualisation
from sotif_uncertainty.visualization import plot_tc_ranking

fig = plot_tc_ranking(tc_results)
plt.show()

### 7.2 Frame-Level Triage (Clause 7)

Flag frames containing high-uncertainty false positives for targeted investigation. This supports the Area 3 -> Area 2 transition (Figure 1).

In [ ]:
from sotif_uncertainty.sotif_analysis import flag_frames, compute_frame_summary

# Flag frames with high-uncertainty FP
frame_flag_results = flag_frames(frame_ids, labels, conf_var, percentile=80.0)

print("=" * 60)
print("FRAME-LEVEL TRIAGE (Clause 7)")
print("=" * 60)
print(f"  Variance threshold (80th pct of FP dist): {frame_flag_results['threshold']:.5f}")
print(f"  Total frames: {frame_flag_results['total_frames']}")
print(f"  Flagged frames: {frame_flag_results['flagged_count']}")
print(f"  Flagged rate: {frame_flag_results['flagged_count']/frame_flag_results['total_frames']:.1%}")
print(f"\n  First 10 flagged frame IDs: {frame_flag_results['flagged_frames'][:10]}")
print(f"\n  These frames are candidates for targeted investigation,")
print(f"  augmentation, or operational restriction.")

# Compute per-frame summaries
frame_summaries = compute_frame_summary(frame_ids, labels, mean_conf, conf_var, conditions)

In [ ]:
# Frame risk scatter (Figure 8)
from sotif_uncertainty.visualization import plot_frame_risk_scatter

fig = plot_frame_risk_scatter(frame_summaries)
plt.show()

### 7.3 ISO 21448 Scenario Categorisation

The SOTIF objective is to reduce the space of unknown hazardous scenarios (Area 3) by identifying triggering conditions and moving them to Area 2 where mitigation measures can be applied.

In [ ]:
from sotif_uncertainty.visualization import plot_iso21448_scenario_grid

fig = plot_iso21448_scenario_grid()
plt.show()

## 8. Complete SOTIF Artefact Summary

Summary of all artefacts produced, mapped to ISO 21448 clauses (Table 5).

In [ ]:
print("=" * 75)
print("SOTIF ARTEFACT SUMMARY (Table 5)")
print("=" * 75)

artefacts = [
    ("Per-detection AUROC, AURC",
     "Performance insufficiency identification (Clause 7)",
     f"AUROC(conf)={disc['auroc_mean_confidence']:.3f}, "
     f"AUROC(var)={disc['auroc_confidence_variance']:.3f}"),
    ("Per-condition FP rate & ranking",
     "Triggering condition identification (Clause 7)",
     f"Top TCs: {tc_results[0]['condition']} ({tc_results[0]['fp_share']:.0%}), "
     f"{tc_results[1]['condition']} ({tc_results[1]['fp_share']:.0%})"),
    ("Frame flags (high-var FP)",
     "Scenario triage: Area 3 -> Area 2 (Clause 7)",
     f"{frame_flag_results['flagged_count']}/{frame_flag_results['total_frames']} frames flagged"),
    ("Coverage & FAR at thresholds",
     "Acceptance criteria input (Clause 11)",
     f"At s>=0.70: cov={[p for p in conf_only_points if abs(p['tau_s']-0.70)<0.01][0]['coverage']:.3f}, FAR={[p for p in conf_only_points if abs(p['tau_s']-0.70)<0.01][0]['far']:.3f}" if any(abs(p['tau_s']-0.70)<0.01 for p in conf_only_points) else "N/A"),
    ("ECE, NLL, Brier",
     "Confidence interpretation (Clause 10)",
     f"ECE={cal['ece']:.3f}, NLL={cal['nll']:.3f}, Brier={cal['brier']:.3f}"),
]

for output, activity, value in artefacts:
    print(f"\n  Output: {output}")
    print(f"  SOTIF Activity: {activity}")
    print(f"  Result: {value}")

## 9. Using Real Data

To run this evaluation on real ensemble outputs:

### Step 1: Train Ensemble (requires GPU)
```bash
# Train 6 SECOND models with different random seeds using OpenPCDet
bash scripts/train_ensemble.sh --seeds 0 1 2 3 4 5
```

### Step 2: Run Ensemble Inference
```bash
python scripts/run_inference.py \
    --cfg_file tools/cfgs/kitti_models/second.yaml \
    --ckpt_dirs output/seed_0 output/seed_1 ... output/seed_5 \
    --data_path data/kitti/testing
```

### Step 3: Load Real Data Instead of Synthetic
```python
import pickle
import numpy as np

# Load ensemble predictions (one pickle per member)
member_preds = []
for seed in range(6):
    with open(f'output/seed_{seed}/result.pkl', 'rb') as f:
        member_preds.append(pickle.load(f))

# Format into scores array (N, K) and boxes array (N, K, 7)
# Then run the same pipeline as above
```

### Available Datasets

| Dataset | Size | Format | Weather | Access |
|---------|------|--------|---------|--------|
| KITTI | ~29 GB | Native | Clear only | Free (cvlibs.net) |
| nuScenes mini | ~4 GB | Convertible | Rain/night | Free (nuscenes.org) |
| MultiFog KITTI | Varies | KITTI | Synthetic fog | Free |
| CADC | ~7K frames | Custom | Snow/winter | Free |
| CARLA (generate) | Custom | KITTI | Any | Free (simulator) |

## 10. Generate All Figures

Save all figures to the `Analysis/` directory.

In [ ]:
from sotif_uncertainty.visualization import generate_all_figures

figures = generate_all_figures(
    metrics=metrics,
    mean_conf=mean_conf,
    conf_var=conf_var,
    labels=labels,
    frame_summaries=frame_summaries,
    tc_results=tc_results,
    operating_points=all_points,
    output_dir='Analysis',
)

print(f"Generated {len(figures)} figures:")
for name in figures:
    print(f"  - Analysis/{name}*.png")
print("\nAll figures saved to Analysis/ directory.")